# Introduction to `ipyleaflet`
Overview: 
    - Create a map usinng basemaps
    - Add a layer to the map using `basemap_to_tiles` method
    - Add a layer to the map using external tile services using, `TileLayer(url=<tiles_url>)`

In [ ]:
%load_ext autoreload
%autoreload 2

In [134]:
# Define an output widget to capture the handler's intermediate printouts
import ipywidgets as widgets
out = widgets.Output(layout={'border': '1px solid black'})
# out

In [34]:
import os, json, requests
from pathlib import Path
# from requests.auth import HTTPBasicAuth

import time, datetime

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
import subprocess
from shapely.geometry import shape, mapping
from planet import api

import ipdb
from pprint import pprint
from IPython.display import JSON, GeoJSON

import ipywidgets as widgets
from ipyleaflet import * 

In [5]:
# Define an output widget to capture the handler's intermediate printouts
import ipywidgets as widgets
out = widgets.Output(layout={'border': '1px solid black'})
# out

In [3]:
def get_mro(someobj):
    mro_tuple = someobj.__class__.__mro__
    display(mro_tuple)
    return mro_tuple

## Create a map

In [41]:
# Map Settings 
# Initial map center in lat/long
paris_latlon = [48.864716, 2.349014] 
jeonju_latlon = [35.8242, 127.1480] 
naju_latlon = [35.032194, 126.715671]

# Initial map zoom level
zoom = 15
colors = {'blue': "#009da5"}

# Set Map Tiles URL
# tile_layer = TileLayer(url= tiles_url)

# Create the map
m = Map(
    center=naju_latlon, 
    zoom=zoom,
)
m.layout={'height': '500px'}
# m.add_layer(tile_layer)

## Show all basemaps

In [51]:
providers = dir(basemaps)
print("Tile Providers: ", providers)

Tile Providers:  ['CartoDB', 'Esri', 'HikeBike', 'Hydda', 'MtbMap', 'NASAGIBS', 'OpenMapSurfer', 'OpenStreetMap', 'OpenTopoMap', 'Stamen', 'Strava']


In [21]:
print(basemaps['CartoDB']['DarkMatter'])
print(basemaps['Esri']['WorldImagery'])

{'url': 'http://c.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}.png', 'max_zoom': 20, 'attribution': '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> &copy; <a href="http://cartodb.com/attributions">CartoDB</a>', 'name': 'CartoDB.DarkMatter'}
{'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', 'max_zoom': 20, 'attribution': 'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community', 'name': 'Esri.WorldImagery'}


In [35]:
# Define an output widget to capture the handler's intermediate printouts
out = widgets.Output(layout={'border': '1px solid black'})
out_marker = widgets.Output(layout={'border': '1px solid black'})

# out

In [101]:
display(out)
display(m)
display(out_marker)

Output(layout=Layout(border='1px solid black'), outputs=({'name': 'stdout', 'text': '=========================…

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Output(layout=Layout(border='1px solid black'))

In [48]:
providers = ['CartoDB', 'Esri', 'HikeBike', 'NASAGIBS', 'OpenStreetMap', 'Strava']
out.clear_output()
with out:
    for provider in providers:
        print("="*80)
        print("Provider: ", provider)
        bmaps = basemaps[provider]
        for bmap_name, bmap_data in bmaps.items():
            print("\t", bmap_name)
            tile_layer = basemap_to_tiles(bmap_data)
            m.add_layer(tile_layer)
            time.sleep(2)

## Set a tile layer


In [114]:
esri_layer = basemap_to_tiles(basemaps.Esri.WorldImagery)
m.clear_layers()
m.add_layer(esri_layer)

WOOHOO!!

In [115]:
# Let's add a market at 금성사
marker = Marker(location=m.center, 
               title="금성사",
               rise_on_hover=True) 
# print(marker)
# get_mro(marker); # Marker 

In [116]:
marker_cb_count = 0
out_marker.clear_output()
@out_marker.capture()
def marker_cb(*args, **kwargs):
    global marker_cb_count 
    marker_cb_count += 1
    print('marker handler called: ', marker_cb_count)
#     print('args: ', args)
    pprint(kwargs)
    
# add the handler to the marker object
marker.on_move(marker_cb)

Marker is a subclass of Layer, so we can use `add_layer` method to add this to the map instance

In [117]:
m.add_layer(marker) #Voila!
# we can check the membership
print(marker in m.layers)

True


In [118]:
# we can change the style of the marker 
marker.interact(opacity=(0,1,0.01))

Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0, step=0.01),))

## Remove layer
When a layer object is added to a map, the layer is assigned a `model.id` attribute. 
Using this information, `remove_layer` method can accept a layer object and remove the object from the map.


In [ ]:
m.remove_layer(marker)

Alternatively, we can remove a layer by getting a layer object at a specific added order


In [82]:
m.remove_layer(m.layers[-1])#remove the last layer (ie. the top-most layer)